In [1]:
import numpy as np
import pandas as pd
import datetime as dt

import matplotlib.pyplot as plt
import plotly.express as px

import yfinance as yf


In [2]:
pd.set_option('display.precision',4)

In [ ]:
def generate_portfolios(returns, numPortfolios, riskFreeRate=0, shortSelling=False):
    tickers = returns.columns
    nAssets = len(tickers)
    mean_returns = returns.mean() * 252
    cov_matrix = returns.cov() * 252


    # Create an empty DataFrame to store the results
    portfolios = pd.DataFrame(columns=[ticker+' weight' for ticker in tickers] + ['Return', 'Risk', 'Sharpe Ratio'], index=range(numPortfolios), dtype=float)

    # Generate random weights and calculate the expected return, volatility and Sharpe ratio
    for i in range(numPortfolios):
        weights = np.random.random(nAssets)
        weights /= np.sum(weights)
        portfolios.loc[i, [ticker+' weight' for ticker in tickers]] = weights

        # Calculate the expected return
        portfolios.loc[i, 'Return'] = np.dot(weights, mean_returns)

        # Calculate the expected volatility
        portfolios.loc[i, 'Risk'] = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

    # Calculate the Sharpe ratio
    portfolios['Sharpe Ratio'] = (portfolios['Return'] - riskFreeRate) / portfolios['Risk']

    return portfolios
    

In [3]:
# Get a list of symbols from FTSEMIB index
ftsemib = pd.read_html('https://en.wikipedia.org/wiki/FTSE_MIB')[1]
ftsemib['ICB Sector'] = ftsemib['ICB Sector'].str.extract(r'\((.*?)\)', expand=False).fillna(ftsemib['ICB Sector'])

ftsemib

,Company,Ticker,ISIN,ICB Sector
0,A2A,A2A.MI,IT0001233417,Electricity
1,Amplifon,AMP.MI,IT0004056880,Health Care
2,Azimut,AZM.MI,IT0003261697,Financial Services
3,Banca Generali,BGN.MI,IT0001031084,Financial Services
4,Banca Mediolanum,BMED.MI,IT0004776628,Financial Services
5,Banco BPM,BAMI.MI,IT0005218380,Banks
6,BPER Banca,BPE.MI,IT0000066123,Banks
7,Buzzi Unicem,BZU.MI,IT0001347308,Industrials
8,Campari,CPR.MI,IT0003849244,Beverages
9,CNH Industrial N.V.,CNHI.MI,NL0010545661,Industrial Goods And Services


In [ ]:
from dash import html, dcc, Input, Output, Dash
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template
import plotly.express as px
import plotly.graph_objects as go
from dash.exceptions import PreventUpdate
import json

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
load_figure_template("CERULEAN")

app = Dash(__name__, external_stylesheets=[dbc.themes.CERULEAN, dbc_css])

app.layout = html.Div([
    dcc.Store(id='store-returns'),
    dcc.Tabs(
        id='tabs',
        value='tab-1',
        className='dbc',
        children=[
            dcc.Tab(
                label='Asset selection',
                id='tab-1',
                className='dbc',
                children=[
                    dbc.Row([
                        dbc.Col(
                            width=3,
                            children=dbc.Card([
                                dcc.Dropdown(
                                    id='ticker-dropdown',
                                    options=[
                                        {'label': f"{row['Company']} ({row['Ticker']})", 'value': row['Ticker']}
                                        for _, row in ftsemib.iterrows()
                                    ],
                                    multi=True,
                                    className='dbc'
                                ),
                                dcc.DatePickerRange(
                                    id='date-picker',
                                    min_date_allowed=dt.date(2010, 1, 1),
                                    max_date_allowed=dt.date.today(),
                                    initial_visible_month=dt.date.today(),
                                    start_date=dt.date(2010, 1, 1),
                                    end_date=dt.date.today(),
                                    className='dbc'
                                ),
                            ])
                        ),
                        
                        dbc.Col(
                            dbc.Card(
                                dcc.Graph(id='markowitz-graph')
                            ),
                        )
                    ])
                ]
            ),
            dcc.Tab(
                id='tab-2',
                label='Monte Carlo Allocation',
                children=[
                    dbc.Row([
                        dbc.Col(width=2,
                            children=[
                                html.Button('Generate',
                                id='generate-button',
                                n_clicks=0,
                                className='dbc')
                        ]),
                        dbc.Col([
                            dcc.Graph(id='mc-portfolios', 
                                      clear_on_unhover=True
                                      )
                        ], width=5),
                        dbc.Col([
                            dcc.Graph(id='portfolio-value',
                                    )
                        ], width=5),
                    ])
                ]
            ),
            dcc.Tab(
                id='tab-3',
                label='Tab 3',
                children=[
                    html.Div('Content tab 3')
                ]
            ),
        ]
    ),
])

@app.callback(
    [Output('markowitz-graph', 'figure'),
    Output('mc-portfolios', 'figure'),
    Output('store-returns', 'data')],
    [Input('ticker-dropdown', 'value'),
    Input('date-picker', 'start_date'),
    Input('date-picker', 'end_date')]
)
def select_assets(tickers, start_date, end_date):
    if not tickers:
        fig = go.Figure().update_xaxes(title='Risk', range=[0, 0.5]).update_yaxes(title='Return', range=[0, 0.4]).update_layout(transition_duration=500)
        returns = pd.DataFrame()
        return fig, fig, returns.to_json()

    start_date = dt.datetime.strptime(start_date, '%Y-%m-%d')
    end_date = dt.datetime.strptime(end_date, '%Y-%m-%d')
    
    try:
        data = yf.download(tickers, start=start_date, end=end_date, )['Adj Close']
    except Exception as e:
        raise PreventUpdate

    returns = data.pct_change().dropna()
    tickers_df = pd.DataFrame({'Return': returns.mean()*252, 'Risk': returns.std()*np.sqrt(252)}, index=tickers).rename_axis('Ticker')

    fig = px.scatter(tickers_df, x='Risk', y='Return', text=tickers_df.index)
    fig.update_traces(textposition='top center').update_layout(transition_duration=500)

    return fig, fig, returns.to_json()

@app.callback(
    [Output('mc-portfolios', 'figure', allow_duplicate=True),
    Output('generate-button', 'n_clicks')],
    [Input('store-returns', 'data'),
    Input('generate-button', 'n_clicks')
    ],
    prevent_initial_call=True
)
def mc_allocation(returns, n_clicks):
    if not n_clicks:
        raise PreventUpdate

    returns = pd.read_json(returns)
    if returns.empty:
        fig = go.Figure().update_layout(transition_duration=500)
        n_clicks = None
        return fig, n_clicks

    n_portfolios = 1000
    mc_portfolios = generate_portfolios(returns, 1000)
    fig = px.scatter(portfolios, x='Risk', y='Return', color='Sharpe Ratio', hover_data={**{ticker +' weight': ':.2f' for ticker in tickers}, **{'Return': ':.2f', 'Risk': ':.2f', 'Sharpe Ratio': ':.2f'}}, opacity=0.5, )

    n_clicks = None

    return fig, n_clicks



app.run_server(debug=True, )

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  2 of 2 completed


## Get list of assets

In [5]:
tickers = ['ENI.MI', 'ENEL.MI', 'STMMI.MI', 'LDO.MI', 'A2A.MI',]
# Sort the tickers
tickers.sort()


In [6]:

start = dt.date(2013, 1, 1)
end = dt.date.today()

data = yf.download(tickers, start=start, end=end)['Adj Close']
data.tail()

[*********************100%***********************]  5 of 5 completed


,A2A.MI,ENEL.MI,ENI.MI,LDO.MI,STMMI.MI
Date,,,,,
2024-02-19,1.6770,5.861,14.170,19.19,41.725
2024-02-20,1.7015,5.903,14.132,19.15,40.935
2024-02-21,1.7025,5.957,14.302,19.03,40.980
2024-02-22,1.6910,5.933,14.230,19.08,42.240
2024-02-23,1.6885,5.958,14.256,19.00,41.530


In [7]:
# Split the data into in-sample and out-of-sample
sample_end = dt.date(2022, 12, 31)
inSample_data = data.loc[:sample_end]
outOfSample_data = data.loc[sample_end:]

In [9]:
# Get the daily returns
returns = inSample_data.pct_change().dropna()

# Collect drifts and standard deviations in the columns of a single DataFrame
tickers_df = pd.DataFrame({'Return': returns.mean() * 252, 'Risk': np.sqrt(np.diag(returns.cov() * 252))}, index=tickers).rename_axis('Ticker')
tickers_df

,Return,Risk
Ticker,,
A2A.MI,0.1807,0.2796
ENEL.MI,0.1224,0.2608
ENI.MI,0.0639,0.2777
LDO.MI,0.1378,0.3795
STMMI.MI,0.2802,0.3853


Add minimum variance line too

In [10]:
# Plot the mean return and standard deviation with plotly
fig = px.scatter(tickers_df, x='Risk', y='Return', text=tickers_df.index)
fig.update_traces(textposition='top center')
fig.update_layout(title='Mean return vs Standard deviation', xaxis_title='Standard deviation', yaxis_title='Mean return')
fig.show()


In [13]:
numPortfolios = 10000
nAssets = len(tickers)

portfolios = generate_portfolios(returns, numPortfolios,)

In [14]:
# Select sample portfolio
sample_portfolio = portfolios.loc[256]
# Determine the performance of the portfolio with out-of-sample data
initialValue = 100
nShares = sample_portfolio[[ticker+' weight' for ticker in tickers]].rename({ticker+' weight' : ticker for ticker in tickers})*initialValue/outOfSample_data.iloc[0]
samplePortfolio_value = nShares.dot(outOfSample_data.T)

In [ ]:
from dash import html, dcc, Input, Output, Dash
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template
import plotly.express as px
import plotly.graph_objects as go
from dash.exceptions import PreventUpdate
import json

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  2 of 2 completed


In [ ]:
app = Dash(__name__,  external_stylesheets=[dbc.themes.CERULEAN, dbc_css])

load_figure_template("CERULEAN")

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='portfolios', 
                      figure=px.scatter(portfolios, x='Risk', y='Return', color='Sharpe Ratio', hover_data={**{ticker +' weight': ':.2f' for ticker in tickers}, **{'Return': ':.2f', 'Risk': ':.2f', 'Sharpe Ratio': ':.2f'}}, opacity=0.5, )
                      .add_scatter(x=tickers_df['Risk'], y=tickers_df['Return'], mode='markers', marker=dict(size=7.5, color='black',),showlegend=False, name='Tickers', text = [f'Asset: {index}<br>Standard deviation: {vol:.2f}<br>Expected return: {ret:.2f}' for index, vol, ret in zip(tickers_df.index, tickers_df['Risk'], tickers_df['Return'])],
                    hoverinfo='text'),
                    clear_on_unhover=True
                      )
        ], width=6),
        dbc.Col([
            dcc.Graph(id='portfolio-value', figure=px.line(samplePortfolio_value, title='Portfolio value over time'),)
        ], width=6),
    ]),
])



@app.callback(
    Output('portfolio-value', 'figure'),
    [Input('portfolios', 'clickData'),
     Input('portfolios', 'hoverData')]
)
def update_portfolio_value(clickData, hoverData):
    if clickData is None:
        # Return an empty figure if no portfolio is selected
        fig = px.line()
    else:
        portfolio = portfolios.loc[clickData['points'][0]['pointIndex']]
        nShares = portfolio[[ticker+' weight' for ticker in tickers]].rename({ticker+' weight' : ticker for ticker in tickers})*initialValue/outOfSample_data.iloc[0]
        portfolio_value = nShares.dot(outOfSample_data.T)
        fig = px.line(portfolio_value).update_layout(showlegend=False, transition_duration=50)
        # y bounds are set to the minimum and maximum value of the portfolio values over all possible portfolios
        ylims = [((initialValue/outOfSample_data.iloc[0])*outOfSample_data.min()).min(), ((initialValue/outOfSample_data.iloc[0])*outOfSample_data.max()).max()]
        fig.update_yaxes(range=ylims)

        if hoverData is not None:    
            if hoverData['points'][0]['curveNumber'] == 0:
                portfolio = portfolios.loc[hoverData['points'][0]['pointIndex']]
                nShares = portfolio[[ticker+' weight' for ticker in tickers]].rename({ticker+' weight' : ticker for ticker in tickers})*initialValue/outOfSample_data.iloc[0]
                portfolio_value = nShares.dot(outOfSample_data.T)
            else:
                ticker = tickers_df.iloc[hoverData['points'][0]['pointIndex']].name
                nShares = initialValue/outOfSample_data.iloc[0][ticker]
                portfolio_value = nShares*outOfSample_data[ticker]
            fig.add_trace(go.Scatter(x=portfolio_value.index, y=portfolio_value, mode='lines',opacity=0.2),)
            fig.update_layout(transition_duration=50, showlegend=False)
    return fig


if __name__ == '__main__':
    app.run_server(debug=True, height=1000, port=8050)

Compare in-sample and out-of-sample returns